# Run from bootstrap paths

Now we will use the initial trajectories we obtained from bootstrapping to run an MSTIS simulation. This will show both how objects can be regenerated from storage and how regenerated equivalent objects can be used in place of objects that weren't stored.

Tasks covered in this notebook:
* Loading OPS objects from storage
* Ways of assigning initial trajectories to initial samples
* Setting up a path sampling simulation with various move schemes
* Visualizing trajectories while the path sampling is running

In [3]:
%matplotlib inline
import openpathsampling as paths
import numpy as np
import math

### Loading things from storage

First we'll reload some of the stuff we stored before. Of course, this starts with opening the file.

In [4]:
old_store = paths.AnalysisStorage("ala_mstis_bootstrap.nc")

A lot of information can be recovered from the old storage, and so we don't have the recreate it. However, we did not save our network, so we'll have to create a new one. Since the network creates the ensembles, that means we will have to translate the trajectories from the old ensembles to new ensembles.

In [5]:
print "PathMovers:", len(old_store.pathmovers)
print "Engines:", len(old_store.engines)
print "Samples:", len(old_store.samples)
print "Trajectories:", len(old_store.trajectories)
print "Ensembles:", len(old_store.ensembles)
print "SampleSets:", len(old_store.samplesets)
print "Snapshots:", len(old_store.snapshots)
print "Networks:", len(old_store.networks)

PathMovers: 18
Engines: 2
Samples: 23
Trajectories: 41
Ensembles: 227
SampleSets: 1
Snapshots: 4092
Networks: 1


Loading from storage is very easy. Each store is a list. We take the 0th snapshot as a template (it doesn't actually matter which one) for the next storage we'll create. There's only one engine stored, so we take the only one.

In [6]:
# template = old_store.snapshots[0]
engine = old_store.engines['default']
mstis = old_store.networks[0]
sset = old_store.tag['sampleset']

In [7]:
sset.sanity_check()

## Running RETIS

Now we run the full calculation. Up to here, we haven't been storing any of our results. This time, we'll start a storage object, and we'll save the network we've created. Then we'll run a new `PathSampling` calculation object.

In [8]:
# logging creates ops_output.log file with details of what the calculation is doing
#import logging.config
#logging.config.fileConfig("logging.conf", disable_existing_loggers=False)

In [7]:
storage = paths.storage.Storage("ala_mstis_production.nc", "w")

In [8]:
storage.snapshots.save(template)

UUID('b1b5ebe6-4ceb-11e6-9e0a-000000000042')

Before we can sample we still need to set the actual `MoveScheme` which determines the
set of moves to apply to our set of samples and effectively doing the steps in
replica (sampleset) space. We pick the default scheme for mstis and feed it with
the engine to be used.

In [9]:
scheme = paths.DefaultScheme(mstis, engine)

and finally generate the `PathSampler` object to conduct the simulation.

In [10]:
mstis_calc = paths.PathSampling(
    storage=storage,
    sample_set=sset,
    move_scheme=scheme
)
mstis_calc.save_frequency = 10

Now everything is ready: let's run the simulation! The first step takes a little since all
necessary information, i.e. the engines, topologies, initial snapshots, ..., need to be
stored. Then the monte carlo steps will be performed.

In [11]:
mstis_calc.run(20)

Working on Monte Carlo cycle number 20
Running for 8 seconds -  2.36 steps per second
Expected time to finish: 0 seconds
DONE! Completed 20 Monte Carlo cycles.


In [12]:
print len(storage.steps)

21


In [13]:
# commented out during development, so we can "run all" and then do more
storage.close()